In [1]:
# Tensorflow 2.3でメモリを指定及び節約して使うためのおまじない。
import tensorflow as tf
physical_devices = tf.config.experimental.list_physical_devices('GPU')
tf.config.set_visible_devices(physical_devices[9], 'GPU')
tf.config.experimental.set_memory_growth(physical_devices[9], True)

In [2]:
import os
dirpath = "/workdir/taki_lab/tiny-imagenet-200/"
train_dir = os.path.join(dirpath, "train")
val_dir = os.path.join(dirpath, "val")
test_dir = os.path.join(dirpath, "test")

In [3]:
from keras.preprocessing.image import ImageDataGenerator
from keras.applications.resnet_v2 import preprocess_input

N_TRAIN = 100000
N_VAL = 10000
INPUT_SIZE = (224, 224)
BATCH_SIZE = 32

train_gen = ImageDataGenerator(preprocessing_function=preprocess_input,
                              rotation_range=10,
                              width_shift_range=0.1,
                              height_shift_range=0.1,
                              shear_range=0.1,
                              zoom_range=0.1,
                              fill_mode="nearest")

val_gen = ImageDataGenerator(preprocessing_function=preprocess_input)

train_generator = train_gen.flow_from_directory(train_dir,
                                               target_size=INPUT_SIZE,
                                               batch_size=BATCH_SIZE,
                                               class_mode="categorical")

val_generator = val_gen.flow_from_directory(val_dir,
                                           target_size=INPUT_SIZE,
                                           batch_size=BATCH_SIZE,
                                           class_mode="categorical")

Found 100000 images belonging to 200 classes.
Found 10000 images belonging to 200 classes.


In [6]:
from keras.models import Model
from keras import layers
from keras import Input
from keras.applications import ResNet50V2
from keras import backend as K
K.clear_session()


resnet50v2_base = ResNet50V2(weights="imagenet", include_top=False,
                             input_tensor=Input(shape=(224,224,3)))

x = resnet50v2_base.output
x = layers.GlobalAveragePooling2D()(x)
x = layers.Dense(1024, activation="relu")(x)
output_tensor = layers.Dense(200, activation="softmax")(x)

model = Model(inputs=resnet50v2_base.input, outputs=output_tensor, name="resnet50v2_ft_da")

trainable_filter = int(len(resnet50v2_base.layers))
for layer in resnet50v2_base.layers[:trainable_filter]:
    layer.trainable = False
    
    if layer.name.startswith("batch_normalization"):
        layer.trainable = True

for layer in resnet50v2_base.layers[trainable_filter:]:
    layer.trainable = True

model.summary()

Model: "resnet50v2_ft_da"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 230, 230, 3)  0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 112, 112, 64) 9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
pool1_pad (ZeroPadding2D)       (None, 114, 114, 64) 0           conv1_conv[0][0]                 
___________________________________________________________________________________

In [ ]:
from keras import optimizers
import wandb
from wandb.keras import WandbCallback
from keras.callbacks import EarlyStopping, ReduceLROnPlateau

# wandbの設定
wandb.init(project="tiny-image-net", name="resnet50v2_ft+da")
config = wandb.config
config.learning_rate = 0.0001
config.lr_factor = 0.5
config.optimaizer = "Adam"
config.batch_size = BATCH_SIZE
config.input_size = INPUT_SIZE
config.rotation_range = 10
config.width_shift_range = 0.1
config.height_shift_range = 0.1
config.shear_range = 0.1
config.zoom_range = 0.1

# 最適化設定
model.compile(optimizer=optimizers.Adam(learning_rate=1e-4),
             loss="categorical_crossentropy",
             metrics=["acc"])

early_stopping = EarlyStopping(monitor="val_acc", patience=5)
reduce_lr = ReduceLROnPlateau(monitor="val_loss", factor=0.5, patience=2, min_lr=1e-5)

callback_list = [early_stopping, reduce_lr, WandbCallback()]


# 学習
history = model.fit(train_generator,
    steps_per_epoch=N_TRAIN//BATCH_SIZE ,
    epochs=1000,
    validation_data=val_generator,
    validation_steps=N_VAL//BATCH_SIZE,
    callbacks=callback_list)

wandb: Wandb version 0.10.2 is available!  To upgrade, please run:
wandb:  $ pip install wandb --upgrade


Epoch 1/1000
3125/3125 [==============================] - 1557s 498ms/step - loss: 2.0756 - acc: 0.5235 - val_loss: 1.4553 - val_acc: 0.6385
Epoch 2/1000
3125/3125 [==============================] - 1553s 497ms/step - loss: 1.5292 - acc: 0.6187 - val_loss: 1.3756 - val_acc: 0.6525
Epoch 3/1000
3125/3125 [==============================] - 1518s 486ms/step - loss: 1.4025 - acc: 0.6451 - val_loss: 1.3609 - val_acc: 0.6551
Epoch 4/1000
3125/3125 [==============================] - 1409s 451ms/step - loss: 1.3056 - acc: 0.6640 - val_loss: 1.3471 - val_acc: 0.6623
Epoch 5/1000
3125/3125 [==============================] - 1417s 453ms/step - loss: 1.2348 - acc: 0.6779 - val_loss: 1.3305 - val_acc: 0.6682
Epoch 6/1000
3125/3125 [==============================] - 1416s 453ms/step - loss: 1.1738 - acc: 0.6903 - val_loss: 1.3332 - val_acc: 0.6695
Epoch 7/1000
3125/3125 [==============================] - 1413s 452ms/step - loss: 1.1249 - acc: 0.7014 - val_loss: 1.3311 - val_acc: 0.6691
Epoch 8/1000
